In [3]:
import os
os.getcwd()

'c:\\Users\\Luis\\AppData\\Local\\Programs\\Microsoft VS Code'

In [6]:
os.chdir("D:\luisf\Documents\Fact")

In [18]:
import ccxt
import time

exchange = ccxt.binance({
    'apiKey': 'WJJYLjoN52IBRCTUJFyi9VPFJC2jSg2rU0YvpXVDykUQDLQ1umYalCre986iDCmS',
    'secret': 'DwyVNXTN5MSN2S49zGrgmpBqIEpEDXtL8xav6rjyyeduqNXhN2mfxC1HPKXdqhSe',
    'enableRateLimit': True,
})

exchange.set_sandbox_mode(True)

# Sincroniza el reloj local con el de Binance
exchange.load_time_difference()

# Luego de sincronizar, puedes hacer la llamada
balance = exchange.fetch_balance()
print(balance)

{'info': {'makerCommission': '0', 'takerCommission': '0', 'buyerCommission': '0', 'sellerCommission': '0', 'commissionRates': {'maker': '0.00000000', 'taker': '0.00000000', 'buyer': '0.00000000', 'seller': '0.00000000'}, 'canTrade': True, 'canWithdraw': True, 'canDeposit': True, 'brokered': False, 'requireSelfTradePrevention': False, 'preventSor': False, 'updateTime': '1748102998621', 'accountType': 'SPOT', 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'}, {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '2022.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '233.00000000', 'locked': '0.00000000'}, {'asset': 'NEO', 'free': '89.00000000', 'locked': '0.00000000'}, {'asset': 'QTUM', 'free': '238.00000000', 'locked': '

In [23]:
# OHLCV = Open, High, Low, Close, Volume
ohlcv = exchange.fetch_ohlcv('BTC/USDT', timeframe='1h', limit=100)

import pandas as pd
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df

,timestamp,open,high,low,close,volume
0,2025-05-20 13:00:00,104784.59,104817.22,104198.57,104461.69,94.92434
1,2025-05-20 14:00:00,104461.69,106910.42,104261.86,104539.88,115.92270
2,2025-05-20 15:00:00,104540.00,105388.45,30000.00,105355.40,110.74340
3,2025-05-20 16:00:00,105349.99,105601.02,25000.00,105212.14,104.38054
4,2025-05-20 17:00:00,105212.50,106320.02,105192.67,106238.43,128.36060
...,...,...,...,...,...,...
95,2025-05-24 12:00:00,109219.46,113674.08,30000.00,108782.61,110.92144
96,2025-05-24 13:00:00,108782.57,113124.03,108539.65,108576.04,74.20715
97,2025-05-24 14:00:00,108576.03,113373.17,33000.00,108959.10,118.57265
98,2025-05-24 15:00:00,108959.10,126488.55,108724.18,108921.42,118.86656


In [24]:
import numpy as np

def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'])

# Condición de compra/venta
df['signal'] = 0
df.loc[df['rsi'] < 30, 'signal'] = 1   # comprar
df.loc[df['rsi'] > 70, 'signal'] = -1  # vender

print(df.tail())


             timestamp       open       high        low      close     volume  \
95 2025-05-24 12:00:00  109219.46  113674.08   30000.00  108782.61  110.92144   
96 2025-05-24 13:00:00  108782.57  113124.03  108539.65  108576.04   74.20715   
97 2025-05-24 14:00:00  108576.03  113373.17   33000.00  108959.10  118.57265   
98 2025-05-24 15:00:00  108959.10  126488.55  108724.18  108921.42  118.86656   
99 2025-05-24 16:00:00  108920.89  108976.96   97927.72  108705.38   22.13438   

          rsi  signal  
95  60.279751       0  
96  64.031193       0  
97  66.493431       0  
98  65.172090       0  
99  59.787086       0  


In [25]:
capital = 1000
position = 0
btc_balance = 0

for i in range(1, len(df)):
    if df.iloc[i]['signal'] == 1 and position == 0:
        btc_balance = capital / df.iloc[i]['close']
        position = 1
        print(f"Compra en {df.iloc[i]['close']:.2f}")
    elif df.iloc[i]['signal'] == -1 and position == 1:
        capital = btc_balance * df.iloc[i]['close']
        btc_balance = 0
        position = 0
        print(f"Venta en {df.iloc[i]['close']:.2f} - Capital: {capital:.2f}")


Compra en 108370.40
